# IBM Call for Code - Generating a realtime map for first responders

In [17]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import folium
import folium.plugins

from trajectories import *

np.random.seed(0)

## Documentation and resources
[folium](https://python-visualization.github.io/folium/plugins.html#folium.plugins.HeatMapWithTime)

[Kaggle folium example with markers](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data)

Not used but very cool: https://blog.prototypr.io/interactive-maps-in-python-part-3-29f14a9b2f7d
https://github.com/vincentropy/python_cartography_tutorial


## Generating emergency calls

In [2]:
# Coordinates
map_center_coordinates = (37.792755, -122.353066) # SF Bay; Berkeley:(37.8716, -122.2727)
map_radius = 1e-3
#firedpt_coordinates = (37.871344, -122.273025) # Berkeley; used for debugging

# Epicenters for simulation
random_epicenters = False
human_epicenters = {'UC Berkeley': (37.873248, -122.259546),
                    #"People's Park": (37.867260, -122.257680),
                    'Ohlone Park': (37.872816, -122.279073),
                    'Presidio': (37.796013, -122.458060),
                    'Golden Gate Park': (37.769054, -122.466374),
                    'Forest Knolls': (37.760245, -122.458546),
                    'Bernal Heights Park': (37.744357, -122.415427),
                    'Nob Hill': (37.792127, -122.412512),
                    'Oakland Park': (37.831823, -122.241672)
                   }

background_epicenters = {'San Francisco': (37.762666, -122.445119),
                         'Oakland': (37.812096, -122.235515),
                         'Berkeley': (37.878219, -122.267008),                        
                        }

human_epicenters_arr = np.array([np.array(v) for (k,v) in human_epicenters.items()])
background_epicenters_arr = np.array([np.array(v) for (k,v) in background_epicenters.items()])

# Simualtion parameters
if random_epicenters:
    nb_epicenters = 10 #use this if you are not setting epicenters
    
simulation_start = datetime.datetime(2019, 4, 27, 5, 32)
simulation_time_delta = 30 #s
simulation_steps = 50 

max_calls = 250

In [3]:
def epicenter_gen(center, radius, nb_points):
    """
    Draw random samples from a multivariate normal distribution with spherical covariance.
    Typical usage: generate specific approximate locations from which calls are coming.
    Input:
        center: distribution mean
        radius: distribution standard deviation
        nb_points: int. number of samples drawn
    Return:
        distribution: np.array
    """
    return np.random.multivariate_normal(center, radius*np.identity(2), nb_points)

def calls_gen(centers, calls_per_center, radius=1e-5):
    """
    Generates a set number of calls taken from a multivariate distribution around each center.
    Input:
        centers: np.array of size (n*2). Coordinates of centers.
        calls_per_center: np.array like of size n. Number of calls for each center.
    Return:
        all_calls: pd.DataFrame. Randomly generated calls.
    """
    n=len(centers)
    all_calls = []
    for i in range(n):
        center_radius = np.random.normal(loc=radius, scale=radius/10)
        center_calls = epicenter_gen(center=centers[i,:], radius=radius, nb_points=calls_per_center[i])
        all_calls.append(center_calls)
    all_calls = np.concatenate(all_calls)
    all_calls = pd.DataFrame(all_calls, columns=['latitude', 'longitude'])
    
    all_calls['time'] = np.random.randint(low=0, high=simulation_steps, size=len(all_calls))
    
    return all_calls


def all_calls_gen(epicenters, background_epicenters=None):
    """
    Generates a set number of calls taken from a multivariate distribution around each center.
    Input:
        centers: np.array of size (n*2). Coordinates of centers.
        calls_per_center: np.array like of size n. Number of calls for each center.
    Return:
        all_calls: pd.DataFrame. Randomly generated calls.
    """
    nb_calls_epicenter = np.random.randint(low=1, high=max_calls, size=len(epicenters))
    all_calls = calls_gen(epicenters, nb_calls_epicenter)
    
    if not background_epicenters is None:
        #print(background_epicenters)
        nb_calls_epicenter = np.random.randint(low=5, high=max_calls, size=len(background_epicenters))
        background_calls = calls_gen(background_epicenters, nb_calls_epicenter, radius=1e-4)
        #print('Background calls:', len(background_calls))
        all_calls = pd.concat([all_calls, background_calls], axis=0)
        
    return all_calls
 
    
if random_epicenters:
    epicenters = epicenter_gen(berkeley_coordinates, map_radius, nb_epicenters)
    background_epicenters = None
else:
    epicenters = human_epicenters_arr
    background_epicenters = background_epicenters_arr

all_calls = all_calls_gen(epicenters, background_epicenters)
all_calls.head()

,latitude,longitude,time
0,37.878055,-122.260467,7
1,37.872827,-122.260093,42
2,37.867677,-122.259823,29
3,37.877570,-122.255987,35
4,37.872113,-122.255686,15


In [4]:
all_calls.sort_values('time').head()

,latitude,longitude,time
496,37.769730,-122.464303,0
776,37.746385,-122.416063,0
781,37.746094,-122.415307,0
803,37.796453,-122.414818,0
142,37.797479,-122.251003,0


## Sending teams on terrain

### Fire department units

#### Creating station dataframe

In [6]:
fire_dpt = pd.read_csv('./mapping/sf_fire_dpt.csv', sep=';', encoding='utf-8', names=['station', 'address'])
fire_dpt.head()

,station,address
0,Station 1,935 Folsom at 5th Street
1,Station 2,1340 Powell Street at Broadway
2,Station 3,1067 Post Street at Polk Street
3,Station 4,449 Mission Rock at 3rd Street
4,Station 5,1301 Turk Street at Webster Street


In [9]:
new_locations = False

import googlemaps
gmaps = googlemaps.Client(key=Gmaps_key)

def get_coordinates(address):
    geocode_result = gmaps.geocode(address)
    location = geocode_result[0]['geometry']['location']
    return location['lat'], location['lng']

if new_locations:
    fire_dpt['latitude'], fire_dpt['longitude'] = zip(*fire_dpt['address'].map(get_coordinates))
    fire_dpt.head()
    fire_dpt.to_csv('./mapping/sf_fire_dpt_coordinates.csv')
else:
    fire_dpt = pd.read_csv('./mapping/sf_fire_dpt_coordinates.csv', index_col=0)

In [10]:
fire_dpt.head()

,station,address,latitude,longitude
0,Station 1,935 Folsom at 5th Street,38.048482,-122.149242
1,Station 2,1340 Powell Street at Broadway,37.797054,-122.410048
2,Station 3,1067 Post Street at Polk Street,37.786630,-122.419318
3,Station 4,449 Mission Rock at 3rd Street,37.772774,-122.389230
4,Station 5,1301 Turk Street at Webster Street,37.780488,-122.430754


#### Simulating station availability

In [11]:
cut_off_lat = 37.736442

fire_dpt['status'] =  np.where(fire_dpt['latitude']<cut_off_lat, 'green', 'red')
fire_dpt['units available'] = np.where(fire_dpt['latitude']<cut_off_lat, np.random.randint(low=2, high=5), 'Unknown')
fire_dpt['units dispatched'] = np.where(fire_dpt['latitude']<cut_off_lat, np.random.randint(low=0, high=3), 'Unknown')
fire_dpt.head()

,station,address,latitude,longitude,status,units available,units dispatched
0,Station 1,935 Folsom at 5th Street,38.048482,-122.149242,red,Unknown,Unknown
1,Station 2,1340 Powell Street at Broadway,37.797054,-122.410048,red,Unknown,Unknown
2,Station 3,1067 Post Street at Polk Street,37.786630,-122.419318,red,Unknown,Unknown
3,Station 4,449 Mission Rock at 3rd Street,37.772774,-122.389230,red,Unknown,Unknown
4,Station 5,1301 Turk Street at Webster Street,37.780488,-122.430754,red,Unknown,Unknown


## Create map

In [14]:
def html_popup(name, address, available, dispatched):
    """
    Generate an html popup for folium.
    Input:
        name: str. Station name
        address: str. Station address
        available: str or int. Number of units available
        dispatched: str or int. Number of units dispatched
        
    Return:
        iframe: folium.IFrame
    """
    html=f"""
        <body style="background-color:#cecedd;">
        <font face="verdana" size=1>
        <h2>{name}</h2>
        <p>
        Address: {address}<br>
        Available units: {available}<br>
        Dispatched units: {dispatched}
        </p>
        </font>
        </body>
        """
    return folium.IFrame(html, width=300, height=100)

In [16]:
def show_er_map(static=True):
    """
    Plot realtime emergency response map using folium. Shows location of fire stations,
    calls and trajectories of dispatched units.
    Input:
        static: bool. Default: True. Set to False to plot dynamic time map.
    Return:
        None
    """
    
    er_map = folium.Map(location=map_center_coordinates,
                        tiles="CartoDB dark_matter",
                        zoom_start=11,
                    )

    # Markers
    """
    folium.features.RegularPolygonMarker(location=firedpt_coordinates,
                                         popup='Berkeley Fire Department',
                                         number_of_sides=4,
                                         radius=5,
                                         fill_color='green'
                                         #icon=folium.Icon(color='red', icon='truck', prefix='fa')
                                        ).add_to(er_map)
    """
    
    SF_stations = folium.FeatureGroup("SF Fire Stations")

    for a in fire_dpt.values:
        station, address, lat, lon, color, available, dispatched = a
        SF_stations.add_child(folium.features.RegularPolygonMarker(location=[lat,lon],
                                                                   popup=folium.Popup(html=html_popup(station, address, available, dispatched), max_width = 300),
                                                                   number_of_sides=4,
                                                                   radius=5,
                                                                   fill_color=color
                                                                  ))
    
    er_map.add_child(SF_stations)
    
    # Static map
    if static:
        # Calls
        heatmap = folium.plugins.HeatMap(all_calls[['latitude', 'longitude']].values, radius=10)
        
        # ER team trajectory
        #folium.PolyLine(test).add_to(er_map)

    # Dynamic map
    else:
        heat_data = [[[row['latitude'],row['longitude']] for _, row in all_calls[all_calls['time'].isin(range(i+1))].iterrows()] for i in range(0,simulation_steps)]
        #start_date = datetime.datetime.now()
        start_date = simulation_start
        names = [(start_date+ i*datetime.timedelta(seconds=simulation_time_delta)).strftime('%D-%H-%M-%S') for i in range(len(heat_data))]
        
        heatmap = folium.plugins.HeatMapWithTime(heat_data, 
                                                 #index=list(call_times), 
                                                 radius = 2,#10,
                                                 auto_play=True,
                                                 index=names
                                                 )

    heatmap.add_to(er_map)
    return er_map

show_er_map(static=False)

In [27]:
from apis import ShortestPath

def closest_id(longitude, latitude, mapper_dict='./mapping/mapper_points.jb', graph=False):
    
    point, metric = '0', 1e9
    dic = joblib.load(mapper_dict)
    
    for point_id, position in zip(dic.keys(), dic.values()):
        mes = np.sum(np.abs(np.asarray(position) - np.asarray([longitude, latitude])))
        if mes < metric: point, metric = point_id, mes
            
    if graph:
        mpr = np.asarray(list(dic.values()))
        plt.figure(figsize=(18,10))
        plt.scatter(mpr[:,0], mpr[:,1], c='wheat', s=2)
        plt.scatter(longitude, latitude, marker='x', s=5, c='black')
        plt.scatter(dic[point][0], dic[point][1], marker='x', s=5, c='orange')
        plt.show()
            
    return point, dic[point]

def display_path(path, mapper='./mapping/mapper_points.jb'):
    
    dic = joblib.load(mapper)
    mpr = np.asarray(list(dic.values()))
    plt.figure(figsize=(18,10))
    plt.scatter(mpr[:,0], mpr[:,1], c='wheat', s=2)
    for point in path:
        plt.scatter(dic[point][0], dic[point][1], marker='d', s=10, c='dodgerblue')
    plt.show()

ini_id, ini_position = closest_id(-122.41, 37.76)
end_id, end_position = closest_id(-122.45, 37.77)

path = ShortestPath(joblib.load('./mapping/map_sf.jb')).dijkstra(ini_id, end_id)

In [71]:
path_to_points

['13023',
 '24560',
 '13022',
 '24559',
 '13021',
 '21665',
 '13020',
 '24558',
 '13019',
 '24557',
 '13018',
 '24556',
 '13017',
 '24555',
 '13016',
 '18438',
 '15929',
 '25135',
 '13015',
 '17142',
 '1156',
 '17109',
 '1113',
 '17076',
 '1111',
 '17116',
 '1108',
 '17113',
 '5793',
 '20819',
 '9430',
 '22163',
 '9431',
 '23000',
 '9432',
 '17685',
 '9433',
 '23001',
 '9434',
 '18935',
 '9435',
 '20402',
 '9436',
 '19331',
 '9437',
 '23002',
 '9438',
 '23003',
 '9439',
 '23004',
 '9440',
 '21743',
 '7318',
 '20595',
 '5502',
 '20596',
 '5503',
 '20597',
 '5504',
 '20598',
 '5505',
 '20599',
 '5506',
 '20600',
 '12096']